In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


#create column names and data frames
column_names = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
species = ['Setosa','Versicolor','Virginica']

train_path = tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv") 
test_path = tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train = pd.read_csv(train_path,names=column_names,header=0)
test = pd.read_csv(test_path,names=column_names,header=0)

train_y = train.pop('Species')
test_y = test.pop('Species')


#create features
features = []

for key in train.keys():
  features.append(tf.feature_column.numeric_column(key=key))




#get input
def get_input(features,label,training=True,batch_num=256):
  ds = tf.data.Dataset.from_tensor_slices((dict(features),label))
  if training:
    ds = ds.shuffle(1000).repeat()

  return ds.batch(batch_num)



#create model
classifier = tf.estimator.DNNClassifier(feature_columns=features,hidden_units=[30,10],n_classes=3)




#train and test
classifier.train(input_fn = lambda: get_input(train,train_y,training=True),steps=6750)
results = classifier.evaluate(input_fn = lambda: get_input(test,test_y,training=False))
print('\n Test set accuracy: {accuracy:0.3f}\n'.format(**results))



#predict
def inputf(features,batch_size=256):
  ds = tf.data.Dataset.from_tensor_slices(dict(features))
  return ds.batch(batch_size) 

inp_features = ['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict = {}

for feature in inp_features:
  valid = True
  while valid:
    val = input(feature + ":")
    if not val.isdigit():
      valid = False
    predict[feature] = [float(val)]

predictions = classifier.predict(lambda: inputf(predict))

for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]
  print("Prediction is '{}' ({:.1f}%)".format(species[class_id],probability * 100))





INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpu9d429q0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:ten